Yes. Import the model and call predict directly:

```python
from pathlib import Path
from serverless_tuner_middleware.model import fit_regression_model, REGRESSION_MODEL

# Use the pre-fit default model
lat_ms = REGRESSION_MODEL.predict(
    "http", "ea1->ea1", "p50", payload_bytes=10*1024, rate_rps=1.0
)

# Or fit from a custom CSV directory, then predict
m = fit_regression_model(Path("middleware/serverless_tuner_middleware/csv"))
lat_ms = m.predict("pubsub", "ea1->we1", "p90", payload_bytes=200*1024, rate_rps=2.0)
print(lat_ms)
```

Unified form with a flexible rate term: lat_q = c_floor_q + a_q / (k_q + rate_rps) + d_rate_q * rate_rps + b_size_q * payload_bytes. Constrain a_q, k_q, b_size_q ≥ 0, but let d_rate_q be signed so HTTP can slope slightly up while Pub/Sub gets the 1/(k+rate) benefit.


In [1]:
from middleware.serverless_tuner_middleware.model import REGRESSION_MODEL
for k,v in REGRESSION_MODEL.coeffs.items():
    print(k, v)

('pubsub', 'us-east1->us-east1', 'p50') Coeffs(a_inv_rate=5.865408051228487, b_size=2.989880066140614e-05, c_floor=2.1252062873468986, d_rate=7.627377713099122, k_shift=0.01)
('http', 'us-east1->us-west1', 'p50') Coeffs(a_inv_rate=10.405549002836011, b_size=0.0004141352149209306, c_floor=4.942788426033644, d_rate=19.523151576519012, k_shift=0.01)
('pubsub', 'us-east1->us-west1', 'p50') Coeffs(a_inv_rate=10.947183770090419, b_size=0.00045125855403260756, c_floor=3.8441811528053904, d_rate=13.610772460699081, k_shift=0.01)
('http', 'us-east1->us-east1', 'p50') Coeffs(a_inv_rate=6.187576644196875, b_size=7.969001825315003e-06, c_floor=2.901354688456081, d_rate=11.41594912391156, k_shift=0.01)
('pubsub', 'us-east1->us-east1', 'p90') Coeffs(a_inv_rate=8.842123539015311, b_size=3.260264670580748e-05, c_floor=3.4713737289580138, d_rate=12.865825617685914, k_shift=0.01)
('http', 'us-east1->us-west1', 'p90') Coeffs(a_inv_rate=11.621247962360798, b_size=0.00044946679383717637, c_floor=5.58135086

In [6]:
from collections import defaultdict
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.stats import (
    compute_edge_samples, aggregate_edge_stats,
    compute_node_samples, aggregate_node_stats,
)
from middleware.serverless_tuner_middleware.critical_path import edge_key
from middleware.serverless_tuner_middleware.rewrite import _infer_region_pair
from middleware.serverless_tuner_middleware.model import REGRESSION_MODEL
from middleware.serverless_tuner_middleware.config import load_config

LOGS = Path("results/tts/logs.ndjson")
CFG = load_config("results/tts/invoker_config.ndjson")

with LOGS.open() as f:
    sends, recvs = parse_events_from_lines(f)

edge_stats = aggregate_edge_stats(compute_edge_samples(sends, recvs))
node_stats = aggregate_node_stats(compute_node_samples(sends, recvs))

payloads = defaultdict(list)
ts_map = defaultdict(list)
for s in sends:
    payloads[edge_key(s)].append(s.payload_size)
    ts_map[edge_key(s)].append(s.ts_ms)

rates = {}
for ek, ts in ts_map.items():
    dur = (max(ts) - min(ts)) / 1000 if len(ts) > 1 else 0
    rates[ek] = len(ts) / dur if dur > 0 else 0.0

print("edge_key | payload_avg/min/max (B) | rate_rps | observed_p50_ms | model_p50_http | model_p50_pubsub")
for e in CFG.edges:
    ek = edge_key(e)
    region = _infer_region_pair(e)
    obs = edge_stats.get((ek, "http")) or edge_stats.get((ek, "pubsub"))
    obs_p50 = obs.p50 if obs else None

    sizes = payloads.get(ek, [])
    avg_payload = sum(sizes) / len(sizes) if sizes else None
    min_payload = min(sizes) if sizes else None
    max_payload = max(sizes) if sizes else None
    rate = rates.get(ek, 0.0)

    http_pred = pubsub_pred = None
    if region and avg_payload is not None:
        http_pred = REGRESSION_MODEL.predict("http", region, "p50", payload_bytes=avg_payload, rate_rps=rate)
        pubsub_pred = REGRESSION_MODEL.predict("pubsub", region, "p50", payload_bytes=avg_payload, rate_rps=rate)

    print(f"{ek} | {avg_payload} / {min_payload} / {max_payload} | {rate:.4f} | {obs_p50} | {http_pred} | {pubsub_pred}")

print("\nnode runtimes (p50):")
for fn, stat in node_stats.items():
    print(f"{fn}: p50={stat.p50} ms count={stat.count}")


edge_key | payload_avg/min/max (B) | rate_rps | observed_p50_ms | model_p50_http | model_p50_pubsub
get_input:entry_point:0->text_2_speech:get_input_0_0:1 | 16009.866310160427 / 16009 / 16010 | 0.0404 | 133.0 | 126.37076332236076 | 119.39477009206409
get_input:entry_point:0->profanity:get_input_0_1:2 | 15997.862637362638 / 15997 / 15998 | 0.0393 | 129.0 | 129.04899532778677 | 121.93673269973729
text_2_speech:get_input_0_0:1->encoding:text_2_speech_1_0:3 | 16012.853846153846 / 16012 / 16013 | 0.0332 | 70.0 | 146.5787659034145 | 138.57346270852946
profanity:get_input_0_1:2->censor:sync: | 15864.852713178294 / 15864 / 15865 | 0.0330 | 71.0 | 147.16370457882283 | 139.1252031017226
encoding:text_2_speech_1_0:3->censor:sync: | None / None / None | 0.0000 | None | None | None

node runtimes (p50):
encoding: p50=1599.0 ms count=75
text_2_speech: p50=7180.0 ms count=84
profanity: p50=41751.0 ms count=9


In [7]:
from collections import defaultdict
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines

with Path("results/tts/logs.ndjson").open() as f:
    sends, recvs = parse_events_from_lines(f)

recvs_by_fn = defaultdict(set)
sends_by_fn = defaultdict(set)
for r in recvs:
    recvs_by_fn[r.fn_name].add(r.run_id)   # run_id only
for s in sends:
    sends_by_fn[s.from_fn].add(s.run_id)   # run_id only

for fn in recvs_by_fn:
    missing = recvs_by_fn[fn] - sends_by_fn[fn]
    print(f"{fn}: recvs={len(recvs_by_fn[fn])}, sends={len(sends_by_fn[fn])}, missing_runtime={len(missing)}")


# Edge sample counts
edge_counts = defaultdict(int)
for s in sends:
    edge_counts[edge_key(s)] += 1
print("\nEdge send counts:")
for ek, cnt in edge_counts.items():
    print(ek, cnt)

profanity: recvs=156, sends=22, missing_runtime=147
text_2_speech: recvs=154, sends=130, missing_runtime=70
censor: recvs=121, sends=0, missing_runtime=121
encoding: recvs=87, sends=107, missing_runtime=12

Edge send counts:
get_input:entry_point:0->text_2_speech:get_input_0_0:1 187
get_input:entry_point:0->profanity:get_input_0_1:2 182
profanity:get_input_0_1:2->censor:sync: 129
text_2_speech:get_input_0_0:1->encoding:text_2_speech_1_0:3 130


In [8]:
from collections import Counter
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/logs.ndjson").open() as f:
    sends, recvs = parse_events_from_lines(f)

send_pairs = Counter((s.run_id, s.taint, s.to_fn) for s in sends if s.from_fn=="profanity")
recv_pairs = Counter((r.run_id, r.taint, r.fn_name) for r in recvs if r.fn_name=="censor")
print("matched sends->recvs:", sum(min(send_pairs[p], recv_pairs.get(p,0)) for p in send_pairs))
print("total sends:", len(send_pairs), "total recvs:", len(recv_pairs))


matched sends->recvs: 21
total sends: 22 total recvs: 121


In [9]:
from pathlib import Path
from collections import Counter
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/logs.ndjson").open() as f:
    sends, _ = parse_events_from_lines(f)

edges = Counter(edge_key(s) for s in sends if s.from_fn == "encoding")
print(edges)


Counter({'profanity:get_input_0_1:2->censor:sync:': 107})


In [10]:
from collections import Counter
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/logs.ndjson").open() as f:
    sends, _ = parse_events_from_lines(f)

by_edge = Counter(edge_key(s) for s in sends)
print(by_edge.get("encoding:text_2_speech_1_0:3->censor:sync:"))  # likely 0
print(by_edge.get("profanity:get_input_0_1:2->censor:sync:"))      # 107


None
129


In [11]:
from collections import Counter
from pathlib import Path
from middleware.serverless_tuner_middleware.logs import parse_events_from_lines
from middleware.serverless_tuner_middleware.critical_path import edge_key

with Path("results/tts/logs.ndjson").open() as f:
    sends, _ = parse_events_from_lines(f)

# All edges emitted by encoding
enc_edges = Counter(edge_key(s) for s in sends if s.from_fn == "encoding")
print("encoding sends:", enc_edges)

# All edges to censor (any source)
censor_edges = Counter(edge_key(s) for s in sends if s.to_fn == "censor")
print("to censor:", censor_edges)


encoding sends: Counter({'profanity:get_input_0_1:2->censor:sync:': 107})
to censor: Counter({'profanity:get_input_0_1:2->censor:sync:': 129})
